In [520]:
import pandas as pd
import numpy as np

## Load data

In [522]:
accidents_kaggle  = pd.read_csv('US_Accidents_Dec20_updated_Kaggle.csv')

In [523]:
fatalities_2016  = pd.read_csv('nhtsa_accidents_fatalities_2016.csv')
# fatalities_2017  = pd.read_csv('nhtsa_accidents_fatalities_2017.csv')
# fatalities_2018  = pd.read_csv('nhtsa_accidents_fatalities_2018.csv')
# fatalities_2019  = pd.read_csv('nhtsa_accidents_fatalities_2019.csv')

In [543]:
fatalities_2016.head()

,STATE,STATENAME,ST_CASE,VE_TOTAL,VE_FORMS,PVH_INVL,PEDS,PERSONS,PERMVIT,PERNOTMVIT,COUNTY,COUNTYNAME,CITY,CITYNAME,DAY,DAYNAME,MONTH,MONTHNAME,YEAR,DAY_WEEK,DAY_WEEKNAME,HOUR,HOURNAME,MINUTE,MINUTENAME,NHS,NHSNAME,ROUTE,ROUTENAME,TWAY_ID,TWAY_ID2,RUR_URB,RUR_URBNAME,FUNC_SYS,FUNC_SYSNAME,RD_OWNER,RD_OWNERNAME,MILEPT,MILEPTNAME,LATITUDE,...,RELJCT2,RELJCT2NAME,TYP_INT,TYP_INTNAME,WRK_ZONE,WRK_ZONENAME,REL_ROAD,REL_ROADNAME,LGT_COND,LGT_CONDNAME,WEATHER1,WEATHER1NAME,WEATHER2,WEATHER2NAME,WEATHER,WEATHERNAME,SCH_BUS,SCH_BUSNAME,RAIL,RAILNAME,NOT_HOUR,NOT_HOURNAME,NOT_MIN,NOT_MINNAME,ARR_HOUR,ARR_HOURNAME,ARR_MIN,ARR_MINNAME,HOSP_HR,HOSP_HRNAME,HOSP_MN,HOSP_MNNAME,CF1,CF1NAME,CF2,CF2NAME,CF3,CF3NAME,FATALS,DRUNK_DR
0,1,Alabama,10001,1,1,0,0,1,1,0,73,JEFFERSON (73),1716,HOOVER,1,1,1,January,2016,6,Friday,9,9:00am-9:59am,25,25,1,This section IS ON the NHS,1,Interstate,I-65,NaN,2,Urban,1,Interstate,1,State Highway Agency,2530,2530,33.426458,...,1,Non-Junction,1,Not an Intersection,0,None,5,Outside Trafficway,1,Daylight,1,Clear,0,No Additional Atmospheric Conditions,1,Clear,0,No,0000000,Not Applicable,9,9:00am-9:59am,31,31,9,9:00am-9:59am,36,36,88,Not Applicable (Not Transported),88,Not Applicable (Not Transported),0,None,0,None,0,None,1,1
1,1,Alabama,10002,1,1,0,0,2,2,0,73,JEFFERSON (73),1716,HOOVER,10,10,1,January,2016,1,Sunday,19,7:00pm-7:59pm,59,59,1,This section IS ON the NHS,1,Interstate,I-65,I-459 RAMP,2,Urban,1,Interstate,1,State Highway Agency,2500,2500,33.400306,...,5,Entrance/Exit Ramp Related,1,Not an Intersection,0,None,4,On Roadside,3,Dark - Lighted,1,Clear,0,No Additional Atmospheric Conditions,1,Clear,0,No,0000000,Not Applicable,99,Unknown,99,Unknown,20,8:00pm-8:59pm,5,5,99,Unknown,99,Unknown EMS Hospital Arrival Time,0,None,0,None,0,None,1,1
2,1,Alabama,10003,2,1,1,0,2,1,1,73,JEFFERSON (73),0,NOT APPLICABLE,1,1,1,January,2016,6,Friday,2,2:00am-2:59am,20,20,1,This section IS ON the NHS,1,Interstate,I-59,NaN,2,Urban,1,Interstate,1,State Highway Agency,1417,1417,33.649244,...,1,Non-Junction,1,Not an Intersection,0,None,4,On Roadside,3,Dark - Lighted,10,Cloudy,0,No Additional Atmospheric Conditions,10,Cloudy,0,No,0000000,Not Applicable,99,Unknown,99,Unknown,2,2:00am-2:59am,31,31,88,Not Applicable (Not Transported),88,Not Applicable (Not Transported),0,None,0,None,0,None,1,0
3,1,Alabama,10004,1,1,0,0,1,1,0,73,JEFFERSON (73),0,NOT APPLICABLE,17,17,1,January,2016,1,Sunday,21,9:00pm-9:59pm,45,45,1,This section IS ON the NHS,1,Interstate,I-65,NaN,2,Urban,1,Interstate,1,State Highway Agency,2543,2543,33.429956,...,1,Non-Junction,1,Not an Intersection,0,None,3,On Median,3,Dark - Lighted,1,Clear,0,No Additional Atmospheric Conditions,1,Clear,0,No,0000000,Not Applicable,99,Unknown,99,Unknown,21,9:00pm-9:59pm,57,57,99,Unknown,99,Unknown EMS Hospital Arrival Time,0,None,0,None,0,None,1,1
4,1,Alabama,10005,1,1,0,0,1,1,0,73,JEFFERSON (73),3125,VESTAVIA HILLS,26,26,1,January,2016,3,Tuesday,20,8:00pm-8:59pm,7,7,1,This section IS ON the NHS,1,Interstate,I-65,NaN,2,Urban,1,Interstate,1,State Highway Agency,2530,2530,33.412050,...,1,Non-Junction,1,Not an Intersection,0,None,5,Outside Trafficway,3,Dark - Lighted,2,Rain,0,No Additional Atmospheric Conditions,2,Rain,0,No,0000000,Not Applicable,20,8:00pm-8:59pm,7,7,20,8:00pm-8:59pm,10,10,88,Not Applicable (Not Transported),88,Not Applicable (Not Transported),0,None,0,None,0,None,1,0


In [524]:
fatalities_2017  = pd.read_csv('nhtsa_accidents_fatalities_2017.csv')


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (40,42) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [525]:
fatalities_2018  = pd.read_csv('nhtsa_accidents_fatalities_2018.csv')


In [526]:
# I was getting an error, and I solved it with this thread: https://stackoverflow.com/questions/18171739/unicodedecodeerror-when-reading-csv-file-in-pandas-with-python
fatalities_2019  = pd.read_csv('nhtsa_accidents_fatalities_2019.csv',encoding = "ISO-8859-1", engine='python')



In [527]:
fatalities_df = pd.concat([fatalities_2016,fatalities_2017])
fatalities_df = pd.concat([fatalities_df,fatalities_2018])
fatalities_df = pd.concat([fatalities_df,fatalities_2019])
fatalities_df.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 136471 entries, 0 to 33243
Data columns (total 91 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   STATE         136471 non-null  int64  
 1   STATENAME     136471 non-null  object 
 2   ST_CASE       136471 non-null  int64  
 3   VE_TOTAL      136471 non-null  int64  
 4   VE_FORMS      136471 non-null  int64  
 5   PVH_INVL      136471 non-null  int64  
 6   PEDS          136471 non-null  int64  
 7   PERSONS       136471 non-null  int64  
 8   PERMVIT       136471 non-null  int64  
 9   PERNOTMVIT    136471 non-null  int64  
 10  COUNTY        136471 non-null  int64  
 11  COUNTYNAME    136471 non-null  object 
 12  CITY          136471 non-null  int64  
 13  CITYNAME      136471 non-null  object 
 14  DAY           136471 non-null  int64  
 15  DAYNAME       136471 non-null  int64  
 16  MONTH         136471 non-null  int64  
 17  MONTHNAME     136471 non-null  object 
 18  YEAR 

We now have two dataFrames, (1) the kaggle dataset with ~1.5M accidents, and the (2) Department of Transportation fatalities dataset.

For each dataset, let's now select specific columns for faster computations:


In [ ]:
# We are only interested in fatalities accidents that were reported to be related to intersections in the official reports.
# 

In [528]:
# Create a new dataset that combines accidents from both intersections: I will revisit these steps in the future. 

fatalities_df.RELJCT2NAME.unique()

array(['Non-Junction', 'Entrance/Exit Ramp Related',
       'Intersection-Related', 'Intersection', 'Driveway Access Related',
       'Through Roadway', 'Other location within Interchange Area',
       'Acceleration/Deceleration Lane', 'Railway Grade Crossing',
       'Entrance/Exit Ramp', 'Crossover-Related', 'Driveway Access',
       'Unknown', 'Shared-Use Path Crossing', 'Not Reported',
       'Reported as Unknown'], dtype=object)

In [529]:
fatalities_intersections = fatalities_df.loc[(fatalities_df['RELJCT2NAME']== 'Intersection-Related') | (fatalities_df['RELJCT2NAME']== 'Intersection')].copy()
fatalities_intersections.count()

STATE        33859
STATENAME    33859
ST_CASE      33859
VE_TOTAL     33859
VE_FORMS     33859
             ...  
CF2NAME      33859
CF3          33859
CF3NAME      33859
FATALS       33859
DRUNK_DR     33859
Length: 91, dtype: int64

In [ ]:
# There are 33,859 accidents with fatalities that happened in intersections or that were intersection-related. 

In [530]:
kaggle_intersections = accidents_kaggle.loc[(accidents_kaggle['Traffic_Signal']== True) | (accidents_kaggle['Crossing']== True) | (accidents_kaggle['Stop']== True)].copy()


Let's now combine both datasets into a single one. The criteria is having the same latititude and longitude coordinates (30m difference) + also having occurred on the same minute. If this happens, then we consider the accident to be the same.

The resulting dataframe will have the following features (columns):
(1) Severity, latitude, longitude, street, city, county, state, zipcode, stop, Traffic_Signal, Crossing, number of fatalities, number of injuries, type of intersection, number of vehicles involved.

In [544]:
# Select the columns of interest in the fatalities_intersections dataset and create a new dataset:

fatalities_df = fatalities_intersections[['STATENAME','COUNTYNAME','CITYNAME','VE_TOTAL','LATITUDE','LONGITUD','DAY','MONTH','YEAR','HOUR','MINUTE','RELJCT2NAME','TYP_INTNAME','FATALS']].copy()

In [545]:
fatalities_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 33859 entries, 10 to 33240
Data columns (total 14 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   STATENAME    33859 non-null  object 
 1   COUNTYNAME   33859 non-null  object 
 2   CITYNAME     33859 non-null  object 
 3   VE_TOTAL     33859 non-null  int64  
 4   LATITUDE     33859 non-null  float64
 5   LONGITUD     33859 non-null  float64
 6   DAY          33859 non-null  int64  
 7   MONTH        33859 non-null  int64  
 8   YEAR         33859 non-null  int64  
 9   HOUR         33859 non-null  int64  
 10  MINUTE       33859 non-null  int64  
 11  RELJCT2NAME  33859 non-null  object 
 12  TYP_INTNAME  33859 non-null  object 
 13  FATALS       33859 non-null  int64  
dtypes: float64(2), int64(7), object(5)
memory usage: 3.9+ MB


In [ ]:
# Merging the two datasets. First, we need to have the date_time for all accidents.


In [546]:
fatalities_df['MONTH'] = fatalities_df['MONTH'].apply(lambda x: str(x).zfill(2))
fatalities_df['DAY'] = fatalities_df['DAY'].apply(lambda x: str(x).zfill(2))
fatalities_df['HOUR'] = fatalities_df['HOUR'].apply(lambda x: str(x).zfill(2))
fatalities_df['MINUTE'] = fatalities_df['MINUTE'].apply(lambda x: str(x).zfill(2))

# Remove null values (which are '99'):
fatalities_df['HOUR'].replace('99', np.NaN,inplace=True)
fatalities_df['MINUTE'].replace('99', np.NaN,inplace=True)

In [547]:
fatalities_df['MINUTE'].unique()

array(['04', '20', '30', '35', '45', '48', '39', '27', '25', '50', '10',
       '28', '40', '33', '42', '47', '00', '26', '56', '05', '23', '34',
       '07', '08', '16', '18', '58', '21', '55', '29', '14', '15', '32',
       '02', '44', '31', '12', '49', '37', '24', '01', '38', '46', '17',
       '51', '36', '57', '22', '19', '11', '59', '06', '43', '09', nan,
       '54', '53', '41', '52', '13', '03'], dtype=object)

In [548]:
import datetime # datetime objects, which allow us to do time-based operations
fatalities_df["DATE_TIME"] =  pd.to_datetime(fatalities_df.YEAR.astype(str) + "/" + fatalities_df.MONTH + "/" + fatalities_df.DAY + " " + fatalities_df.HOUR + ":" + fatalities_df.MINUTE, format="%Y/%m/%d %H:%M")

In [549]:
# Now we have a new column 'DATE_TIME' that informs us about when the fatality accident happened. Pretty good!

# Delete the YEAR, DAY, MONTH, HOUR, MINUTE columns.
fatalities_df.drop(columns=['YEAR','DAY','MONTH','HOUR','MINUTE'],inplace = True)
# Rename columns:
fatalities_df.rename(columns = {'STATENAME':'State','COUNTYNAME':'County','CITYNAME':'City','LATITUDE': 'Latitude','LONGITUD':'Longitude','RELJCT2NAME':'Intersection_status','TYP_INTNAME':'Intersection_type','FATALS': 'number_of_fatalities','VE_TOTAL':'num_vehicles'},inplace=True)
# Great! We now do have a cleaned fatalities dataset. Just accidents at intersections, with their locations, time, and number of fatalities (always at leat 1 fatality)

In [550]:
fatalities_df['origin_dataset'] = 'fatalities_reported'
fatalities_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 33859 entries, 10 to 33240
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   State                 33859 non-null  object        
 1   County                33859 non-null  object        
 2   City                  33859 non-null  object        
 3   num_vehicles          33859 non-null  int64         
 4   Latitude              33859 non-null  float64       
 5   Longitude             33859 non-null  float64       
 6   Intersection_status   33859 non-null  object        
 7   Intersection_type     33859 non-null  object        
 8   number_of_fatalities  33859 non-null  int64         
 9   DATE_TIME             33773 non-null  datetime64[ns]
 10  origin_dataset        33859 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(2), object(6)
memory usage: 3.1+ MB


In [552]:
# Now, let's pre-process the Kaggle dataset in order to have it in the same format:

# Remove duplicates in kaggle_intersections:
kaggle_intersections.drop_duplicates(subset=['Start_Time', 'End_Time','Start_Lat','Start_Lng'],inplace=True)

# This selects the columns of interest: 
kaggle_df = kaggle_intersections[['State','County','City','Start_Time','Start_Lat','Start_Lng','Crossing','Traffic_Signal','Stop']].copy()

# Rename columns:
kaggle_df.rename(columns = {'Start_Time':'DATE_TIME','Start_Lat': 'Latitude','Start_Lng':'Longitude'},inplace=True)
kaggle_df['origin_dataset'] = 'no_fatalities_information'
kaggle_df.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 185680 entries, 6 to 1516018
Data columns (total 10 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   State           185680 non-null  object 
 1   County          185680 non-null  object 
 2   City            185668 non-null  object 
 3   DATE_TIME       185680 non-null  object 
 4   Latitude        185680 non-null  float64
 5   Longitude       185680 non-null  float64
 6   Crossing        185680 non-null  bool   
 7   Traffic_Signal  185680 non-null  bool   
 8   Stop            185680 non-null  bool   
 9   origin_dataset  185680 non-null  object 
dtypes: bool(3), float64(2), object(5)
memory usage: 11.9+ MB


In [569]:
# Now, let's merge both datasets!
all_accidents_df = pd.concat([fatalities_df,kaggle_df],ignore_index=True)
all_accidents_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 219539 entries, 0 to 219538
Data columns (total 14 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   State                 219539 non-null  object 
 1   County                219539 non-null  object 
 2   City                  219527 non-null  object 
 3   num_vehicles          33859 non-null   float64
 4   Latitude              219539 non-null  float64
 5   Longitude             219539 non-null  float64
 6   Intersection_status   33859 non-null   object 
 7   Intersection_type     33859 non-null   object 
 8   number_of_fatalities  33859 non-null   float64
 9   DATE_TIME             219453 non-null  object 
 10  origin_dataset        219539 non-null  object 
 11  Crossing              185680 non-null  object 
 12  Traffic_Signal        185680 non-null  object 
 13  Stop                  185680 non-null  object 
dtypes: float64(4), object(10)
memory usage: 23.4+ MB


In [575]:
all_accidents_df.head()

,State,County,City,num_vehicles,Latitude,Longitude,Intersection_status,Intersection_type,number_of_fatalities,DATE_TIME,origin_dataset,Crossing,Traffic_Signal,Stop
0,Alabama,JEFFERSON,BIRMINGHAM,2.0,33.531136,-86.757786,Intersection-Related,Four-Way Intersection,1.0,2016-01-06 08:04:00,fatalities_reported,NaN,NaN,NaN
1,Alabama,TALLAPOOSA,CAMP HILL,3.0,32.800489,-85.644958,Intersection,Four-Way Intersection,4.0,2016-01-10 01:20:00,fatalities_reported,NaN,NaN,NaN
2,Alabama,LAWRENCE,NOT APPLICABLE,2.0,34.416869,-87.161528,Intersection,Four-Way Intersection,1.0,2016-01-13 07:30:00,fatalities_reported,NaN,NaN,NaN
3,Alabama,JEFFERSON,BIRMINGHAM,4.0,33.513639,-86.792417,Intersection,Four-Way Intersection,1.0,2016-01-14 10:35:00,fatalities_reported,NaN,NaN,NaN
4,Alabama,CALHOUN,NOT APPLICABLE,2.0,33.886472,-85.916733,Intersection,Four-Way Intersection,1.0,2016-01-15 21:45:00,fatalities_reported,NaN,NaN,NaN


In [571]:
# Let's remove the last parenthesis (and its contents) for the counties:
import re
all_accidents_df['County'] = all_accidents_df['County'].apply(lambda x: re.sub(r"\(.*\)", "", x))


In [572]:
# Now, let's convert states from shorter notation to the whole state name:
def converts_state_names(row):
    us_state_to_abbrev = {
    "Alabama": "AL",
    "Alaska": "AK",
    "Arizona": "AZ",
    "Arkansas": "AR",
    "California": "CA",
    "Colorado": "CO",
    "Connecticut": "CT",
    "Delaware": "DE",
    "Florida": "FL",
    "Georgia": "GA",
    "Hawaii": "HI",
    "Idaho": "ID",
    "Illinois": "IL",
    "Indiana": "IN",
    "Iowa": "IA",
    "Kansas": "KS",
    "Kentucky": "KY",
    "Louisiana": "LA",
    "Maine": "ME",
    "Maryland": "MD",
    "Massachusetts": "MA",
    "Michigan": "MI",
    "Minnesota": "MN",
    "Mississippi": "MS",
    "Missouri": "MO",
    "Montana": "MT",
    "Nebraska": "NE",
    "Nevada": "NV",
    "New Hampshire": "NH",
    "New Jersey": "NJ",
    "New Mexico": "NM",
    "New York": "NY",
    "North Carolina": "NC",
    "North Dakota": "ND",
    "Ohio": "OH",
    "Oklahoma": "OK",
    "Oregon": "OR",
    "Pennsylvania": "PA",
    "Rhode Island": "RI",
    "South Carolina": "SC",
    "South Dakota": "SD",
    "Tennessee": "TN",
    "Texas": "TX",
    "Utah": "UT",
    "Vermont": "VT",
    "Virginia": "VA",
    "Washington": "WA",
    "West Virginia": "WV",
    "Wisconsin": "WI",
    "Wyoming": "WY",
    "District of Columbia": "DC",
    "American Samoa": "AS",
    "Guam": "GU",
    "Northern Mariana Islands": "MP",
    "Puerto Rico": "PR",
    "United States Minor Outlying Islands": "UM",
    "U.S. Virgin Islands": "VI"}
    
    # invert the dictionary
    abbrev_to_us_state = dict(map(reversed, us_state_to_abbrev.items()))
    if len(row) == 2:
        return abbrev_to_us_state.get(row)
    else:
      return row


all_accidents_df['State'] = all_accidents_df['State'].apply(converts_state_names)



In [576]:
all_accidents_df.reset_index(inplace=True,drop=True)

In [561]:
# We have now our merged dataset. We now will run an custom algorithm to find clusters (defined as accidents happening and less than 50m):


In [72]:
# This is the haversine distance calculation. It will allow us to find the distance between two given accident locations:
from math import sin, cos, sqrt, atan2, radians

def distance_between_two_locations(latitude_1,longitude_1,latitude_2,longitude_2):
    # approximate radius of earth in km
    R = 6373.0
    latitude_1 = radians(latitude_1)
    latitude_2 = radians(latitude_2)
    longitude_1 = radians(longitude_1)
    longitude_2 = radians(longitude_2)

    dlatitude = latitude_2 - latitude_1
    dlongitude = longitude_2 - longitude_1
    a = sin(dlatitude / 2)**2 + cos(latitude_1) * cos(latitude_2) * sin(dlongitude / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    distance = R * c * 1000 # Expressed in meters.
    
    return distance

In [ ]:
global all_accidents_df

In [ ]:
all_accidents_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 219539 entries, 0 to 219538
Data columns (total 13 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   State                 219539 non-null  object 
 1   num_vehicles          33859 non-null   float64
 2   Latitude              219539 non-null  float64
 3   Longitude             219539 non-null  float64
 4   Intersection_status   33859 non-null   object 
 5   Intersection_type     33859 non-null   object 
 6   number_of_fatalities  33859 non-null   float64
 7   DATE_TIME             219453 non-null  object 
 8   origin_dataset        219539 non-null  object 
 9   City                  185668 non-null  object 
 10  Crossing              185680 non-null  object 
 11  Traffic_Signal        185680 non-null  object 
 12  Stop                  185680 non-null  object 
dtypes: float64(4), object(9)
memory usage: 21.8+ MB


In [66]:
def finds_clusters(row):
    # Find latitude and longitude of the current accident

    lat,lon = all_accidents_df[['Latitude','Longitude']].iloc[row]
    # Filter all accidents above or below an absolute difference of 0.01 for the latitude and longitude. 
    # For the current accident, this will filter out all the accidents that occurred more than ~1000-12000m away. 
    narrowed_list_accidents = all_accidents_df.loc[(abs(all_accidents_df['Latitude'] - lat) < 0.01) & (abs(all_accidents_df['Longitude'] - lon) < 0.01)][['Latitude','Longitude']].copy() # & ~all_accidents_df.index.isin([0])
    narrowed_list_accidents['distance'] = [distance_between_two_locations(lat,lon,narrowed_list_accidents['Latitude'].iloc[i],narrowed_list_accidents['Longitude'].iloc[i]) for i in range(len(narrowed_list_accidents))]

    # And now, we will find which accidents occurred within 50 meters, using the Haversine distance.
    cluster_accidents = list(narrowed_list_accidents[narrowed_list_accidents['distance'] <= 50].index)
    if row%1000 == 0:
        print(row)
    del narrowed_list_accidents,lat,lon

    
    if cluster_accidents:
        if len(cluster_accidents) > 1:
            return cluster_accidents
        # else: # in case there was only one accident (itself), we want to still keep the accident information for web app visualization:
        #     # if all_accidents_df['origin_dataset'].iloc[row] == 'fatalities_reported':
        #     return [row]

    



In [643]:
import pickle
path = '/content/drive/MyDrive/datasets_pickle/'
pickle_out = open(path + "all_accidents_df.pickle", "wb")
pickle.dump(all_accidents_df,pickle_out)
pickle_out.close()

In [ ]:
list_of_clusters = []
for accident in range(len(all_accidents_df)):
    list_of_clusters.append(finds_clusters(accident))

path = '/content/drive/MyDrive/datasets_pickle/'
pickle_out = open(path + "list_of_clusters.pickle", "wb")
pickle.dump(list_of_clusters,pickle_out)
pickle_out.close()


In [5]:
# Load pickle from google drive

import pickle

path = '/content/all_accidents_df.pickle'
with open(path,'rb') as read_file:
    all_accidents_df = pickle.load(read_file)

path = '/content/list_of_clusters.pickle'
with open(path,'rb') as read_file:
    list_of_clusters = pickle.load(read_file)


In [77]:
# Using list comprehension
# to remove None values in list
list_of_clusters = [i for i in list_of_clusters if i]

In [82]:
# Finds unique lists of accidents and outputs a numpy array with the count for each unique list, and another array with the accident lists themselves:
# Generate numpy arrays with the unique accident lists and count.
unique_accidents_lists, count_unique_accidents_lists = np.unique(list_of_clusters, return_counts=True)

/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:136: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order, subok=True)


In [620]:
# Creates a new dataFrame, by combining the two numpy arrays above:
dataset_clusters = pd.DataFrame({'count': count_unique_accidents_lists, 'accidents': list(unique_accidents_lists)}, columns=['count', 'accidents'])


In [621]:
dataset_clusters.reset_index(drop=True,inplace=True)


In [622]:
# Converts all lists into the set type:
dataset_clusters['accidents'] = dataset_clusters['accidents'].apply(set)

In [623]:
# Reorder in descending order
dataset_clusters.sort_values('count', ascending = False, inplace = True)
dataset_clusters.reset_index(drop=True,inplace=True)


In [624]:
from copy import deepcopy
dataset_clusters_backup = deepcopy(dataset_clusters)

In [625]:
# New column, reflecting the status of each cluster (we initially set all clusters as 'not-evaluated')

dataset_clusters=dataset_clusters.assign(cluster_status='not_evaluated')
dataset_clusters.head()

,count,accidents,cluster_status
0,175,"{171521, 182785, 96776, 182284, 68111, 188945,...",not_evaluated
1,117,"{124419, 53771, 53772, 206350, 206351, 45077, ...",not_evaluated
2,101,"{48641, 155144, 51721, 155147, 155148, 155151,...",not_evaluated
3,92,"{183808, 182810, 181278, 174111, 211497, 13162...",not_evaluated
4,91,"{175104, 217602, 217603, 50181, 186385, 199735...",not_evaluated


In [626]:
# Loop through each cluster: 
for i in range(len(dataset_clusters)):
    if i % 1000 == 0:
      print(i)
    # But only consider those that have not yet been evaluated:
    if dataset_clusters['cluster_status'].iloc[i] == 'not_evaluated':
        # The current cluster (as dataset_clusters is sorted in descending order) is the most repeated cluster of those possible around an intersection.
        # If there are clusters (sets) down the road (>i) that share some accidents with the current cluster, such clusters are not the core for the 
        # intersection as they were observed fewer times. Let's label the current cluster as a 'core_cluster'.
        dataset_clusters.at[i, 'cluster_status'] = 'core_cluster'

        # In the next lines, we are going to find those clusters and label them as 'not_core_cluster':

        # Store the current cluster for subsequent comparisons with other clusters
        current_cluster = dataset_clusters['accidents'].iloc[i]

        # In order to reduce computational complexity, we
        # filter the clusters that have not yet been evaluated in any loop and that are > i
        filtered_clusters_df = dataset_clusters.loc[(dataset_clusters.index > i) & (dataset_clusters['cluster_status'] == 'not_evaluated')].copy()
        # The following list comprehension outputs the positions (indexes) in filtered_clusters_df of the clusters that intersect with the current cluster. 
        indexes_clusters_intersecting = [filtered_clusters_df.index[cluster_index] for cluster_index,cluster_candidate in enumerate(filtered_clusters_df['accidents']) if current_cluster.intersection(cluster_candidate)]
        for idx in indexes_clusters_intersecting:
            dataset_clusters.at[idx, 'cluster_status'] = 'not_core_cluster'
        del current_cluster,filtered_clusters_df,indexes_clusters_intersecting

    # Get only indices of those that have intersection. 
    # map these indices back to the original one. 

   


0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000


In [627]:
from copy import deepcopy
dataset_clusters_backup = deepcopy(dataset_clusters)

In [628]:
path = '/content/drive/MyDrive/datasets_pickle/'
pickle_out = open(path + "dataset_clusters.pickle", "wb")
pickle.dump(dataset_clusters,pickle_out)
pickle_out.close()


In [629]:
dataset_clusters['cluster_status'].value_counts()

core_cluster        31445
not_core_cluster     3244
Name: cluster_status, dtype: int64

In [630]:
all_accidents_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 219539 entries, 0 to 219538
Data columns (total 14 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   State                 219539 non-null  object 
 1   County                219539 non-null  object 
 2   City                  219527 non-null  object 
 3   num_vehicles          33859 non-null   float64
 4   Latitude              219539 non-null  float64
 5   Longitude             219539 non-null  float64
 6   Intersection_status   33859 non-null   object 
 7   Intersection_type     33859 non-null   object 
 8   number_of_fatalities  33859 non-null   float64
 9   DATE_TIME             219453 non-null  object 
 10  origin_dataset        219539 non-null  object 
 11  Crossing              185680 non-null  object 
 12  Traffic_Signal        185680 non-null  object 
 13  Stop                  185680 non-null  object 
dtypes: float64(4), object(10)
memory usage: 23.4+ MB


In [631]:
# Probably this step is unnecessary:

dataset_clusters=dataset_clusters.assign(Latitude='not_available')
dataset_clusters=dataset_clusters.assign(Longitude='not_available')



In [632]:
# Now, let's output a simple dataset that allows us to make a fast plot with plotly.

for i in range(len(dataset_clusters)):
    current_accidents = list(dataset_clusters['accidents'].iloc[i])
    dataset_clusters.at[i,'Latitude'] = np.mean([all_accidents_df['Latitude'].iloc[accident_ID] for accident_ID in current_accidents])
    dataset_clusters.at[i,'Longitude'] = np.mean([all_accidents_df['Longitude'].iloc[accident_ID] for accident_ID in current_accidents])
    dataset_clusters.at[i,'num_accidents_fatalities'] = np.sum([1 for accident_ID in current_accidents if all_accidents_df['origin_dataset'].iloc[accident_ID] == 'fatalities_reported'])


In [633]:
# Checking if all clusters were assigned latitude and longitude coordinates. 
(dataset_clusters['Latitude'] == 'not_available').any(),(dataset_clusters['Longitude'] == 'not_available').any()

(False, False)

In [634]:
dataset_clusters['num_accidents_fatalities'].max()

19.0

In [ ]:
# Add the ratio num_fatalities/count in a new column. 

In [635]:
# Remove clusters that are not core clusters:
dataset_clusters = dataset_clusters[dataset_clusters['cluster_status']=='core_cluster']
dataset_clusters

,count,accidents,cluster_status,Latitude,Longitude,num_accidents_fatalities
0,175,"{171521, 182785, 96776, 182284, 68111, 188945,...",core_cluster,37.8083,-122.367,0.0
1,117,"{124419, 53771, 53772, 206350, 206351, 45077, ...",core_cluster,35.2025,-80.726,0.0
2,101,"{48641, 155144, 51721, 155147, 155148, 155151,...",core_cluster,37.1702,-121.99,0.0
3,92,"{183808, 182810, 181278, 174111, 211497, 13162...",core_cluster,34.0099,-117.823,0.0
4,91,"{175104, 217602, 217603, 50181, 186385, 199735...",core_cluster,38.1552,-121.676,0.0
...,...,...,...,...,...,...
34639,1,"{144899, 26215}",core_cluster,37.7844,-122.473,1.0
34645,1,"{66512, 70448, 101320, 82991}",core_cluster,32.6747,-96.8223,0.0
34647,1,"{76900, 201013}",core_cluster,39.0517,-77.0751,0.0
34670,1,"{46468, 46469}",core_cluster,41.9002,-87.6242,0.0


4

In [646]:
# There is a problem with the counts that I will have to review later. In the meantime, I am going to take as a reference the number of accidents in a cluster and not the official count. 
dataset_clusters['count'] = dataset_clusters['accidents'].apply(lambda x: len(x))

In [638]:
# Let's store now, for each intersection, the city, county and 
dataset_clusters.reset_index(drop=True,inplace=True)


In [640]:
from copy import deepcopy
dataset_clusters_backup = deepcopy(dataset_clusters)

In [641]:
for i,cluster in enumerate(dataset_clusters['accidents']):
  dataset_clusters.at[i,'State'] = all_accidents_df['State'].iloc[list(cluster)[0]] 
  dataset_clusters.at[i,'County'] = all_accidents_df['County'].iloc[list(cluster)[0]] 
  dataset_clusters.at[i,'City'] = all_accidents_df['City'].iloc[list(cluster)[0]] 


  

In [652]:
del dataset_clusters['cluster_status'] # There must be a better way to do it. 

In [645]:
# This creates a new column with the fatalities ratio

dataset_clusters['fatalities_ratio'] = dataset_clusters['num_accidents_fatalities']/dataset_clusters['count_updated']

In [654]:
# Save dataset_clusters:

dataset_clusters.to_csv('dataset_clusters.csv', index = True) # Removes index and header for insertion into SQL databse

In [653]:
dataset_clusters

,count,accidents,Latitude,Longitude,num_accidents_fatalities,State,County,City,fatalities_ratio
0,175,"{171521, 182785, 96776, 182284, 68111, 188945,...",37.8083,-122.367,0.0,California,San Francisco,San Francisco,0.0
1,117,"{124419, 53771, 53772, 206350, 206351, 45077, ...",35.2025,-80.726,0.0,North Carolina,Mecklenburg,Charlotte,0.0
2,101,"{48641, 155144, 51721, 155147, 155148, 155151,...",37.1702,-121.99,0.0,California,Santa Clara,Los Gatos,0.0
3,98,"{183808, 182810, 181278, 174111, 211497, 13162...",34.0099,-117.823,0.0,California,Los Angeles,Diamond Bar,0.0
4,91,"{175104, 217602, 217603, 50181, 186385, 199735...",38.1552,-121.676,0.0,California,Sacramento,Isleton,0.0
...,...,...,...,...,...,...,...,...,...
31440,2,"{144899, 26215}",37.7844,-122.473,1.0,California,San Francisco,San Francisco,0.5
31441,4,"{66512, 70448, 101320, 82991}",32.6747,-96.8223,0.0,Texas,Dallas,Dallas,0.0
31442,2,"{76900, 201013}",39.0517,-77.0751,0.0,Maryland,Montgomery,Silver Spring,0.0
31443,2,"{46468, 46469}",41.9002,-87.6242,0.0,Illinois,Cook,Chicago,0.0


In [ ]:
# To avoid disconnection:
while True:pass